<a href="https://colab.research.google.com/github/iskra3138/GluonTS/blob/master/Electricity_DeepAR_DeepFactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경 구현

Colab에서 어떤 GPU가 할당되었는 지 확인

In [0]:
!nvidia-smi

Fri May 22 01:10:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

MXNET 설치 
- GluonTS는 Amazon이 2019년 공개한 Tool로서 MXNET을 backbone으로 사용
  - https://aws.amazon.com/blogs/machine-learning/creating-neural-time-series-models-with-gluon-time-series/

- 어떤 버전의 MXNET을 설치해야 하는 지는 아래 링크 참고
  - https://mxnet.apache.org/get_started/?platform=linux&language=python&processor=gpu&environ=pip&

In [0]:
# nvidia-smi로 CUDA Version 확인하고 해당되는 MXNET 설치
!pip install mxnet-cu101

     |████████████████████████████████| 710.5MB 26kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


GluonTS를 설치함

In [0]:
!pip install gluonts

     |████████████████████████████████| 419kB 8.5MB/s 
     |████████████████████████████████| 194kB 20.4MB/s 
     |████████████████████████████████| 7.4MB 24.5MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68028 sha256=4f42bcb003f3282444347fad1bd2e13038c45df5caf9905c1e7ec48ed7c9bf5d
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson


## Dataset Loading

In [0]:
# Third-party imports
%matplotlib inline
import mxnet as mx
from mxnet import gluon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from itertools import islice
from pathlib import Path

In [0]:
mx.random.seed(0)
np.random.seed(0)

In [0]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

In [0]:
print(f"Available datasets: {list(dataset_recipes.keys())}")

Available datasets: ['constant', 'exchange_rate', 'solar-energy', 'electricity', 'traffic', 'exchange_rate_nips', 'electricity_nips', 'traffic_nips', 'solar_nips', 'wiki-rolling_nips', 'taxi_30min', 'm4_hourly', 'm4_daily', 'm4_weekly', 'm4_monthly', 'm4_quarterly', 'm4_yearly', 'm5']


### Electricity 데이터 확인

In [0]:
dataset = get_dataset("electricity", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/test/data.json


train data entry의 각 Field 확인 추가

In [0]:
# get the first time series in the training set
# training dataset의 첫번째 entry를 보겠음
### entry에는 start field, target field, feat_statc_cat filed, source filed가 있음
train_entry = next(iter(dataset.train))
train_entry.keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# YJ
## train entry 개수(데이터 개수) 확인
i = 0
for _ in iter(dataset.train):
  i+=1
print (i)

321


In [0]:
# YJ
## train entry 개수(데이터 개수) 확인
for train_entry in iter(dataset.train):
  print (train_entry['feat_static_cat'])

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[40]
[41]
[42]
[43]
[44]
[45]
[46]
[47]
[48]
[49]
[50]
[51]
[52]
[53]
[54]
[55]
[56]
[57]
[58]
[59]
[60]
[61]
[62]
[63]
[64]
[65]
[66]
[67]
[68]
[69]
[70]
[71]
[72]
[73]
[74]
[75]
[76]
[77]
[78]
[79]
[80]
[81]
[82]
[83]
[84]
[85]
[86]
[87]
[88]
[89]
[90]
[91]
[92]
[93]
[94]
[95]
[96]
[97]
[98]
[99]
[100]
[101]
[102]
[103]
[104]
[105]
[106]
[107]
[108]
[109]
[110]
[111]
[112]
[113]
[114]
[115]
[116]
[117]
[118]
[119]
[120]
[121]
[122]
[123]
[124]
[125]
[126]
[127]
[128]
[129]
[130]
[131]
[132]
[133]
[134]
[135]
[136]
[137]
[138]
[139]
[140]
[141]
[142]
[143]
[144]
[145]
[146]
[147]
[148]
[149]
[150]
[151]
[152]
[153]
[154]
[155]
[156]
[157]
[158]
[159]
[160]
[161]
[162]
[163]
[164]
[165]
[166]
[167]
[168]
[169]
[170]
[171]
[172]
[173]
[174]
[175]
[176]
[177]
[178]
[179]
[180]
[181]
[182]
[183]
[184]


In [0]:
# YJ
## Start filed 확인
print (train_entry['start'])

2012-01-01 00:00:00


In [0]:
# YJ
## target filed 길이 확인(time serise 길이)
print (train_entry['target'].shape)

(21044,)


In [0]:
# YJ
## target field target value 앞뒤로 10개씩 확인
print (train_entry['target'][:10])
print (train_entry['target'][-10:])

[14. 18. 21. 20. 22. 20. 20. 20. 13. 11.]
[6. 4. 4. 2. 2. 4. 8. 6. 9. 7.]


In [0]:
# YJ
## 'feat_static_cat' field 확인
## 'feat_static_cat' field는 일단 여기서 무시하고 넘어가고 나중에 다루겠다 함
print (train_entry['feat_static_cat'])

[0]


In [0]:
# YJ
## source field 확인
## source는 데이터가 저장되어 있는 경로에 대한 정보를 담고 있는 것 같음
print (train_entry['source'])

SourceContext(source=PosixPath('/root/.mxnet/gluon-ts/datasets/electricity/train/data.json'), row=0)


We can similarly examine the first entry of the test dataset. We should expect exactly the same fields as in the train dataset.

##### test data entry의 각 Field 확인 추가

In [0]:
# get the first time series in the test set
test_entry = next(iter(dataset.test))
test_entry.keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# YJ
## test entry 개수(데이터 개수) 확인
### train entry 개수(414)와 같음
i = 0
for _ in iter(dataset.test):
  i+=1
print (i)

2247


In [0]:
# YJ
## 'start' field 확인
### train data, test data 동일
print (train_entry['start'])
print (test_entry['start'])

2012-01-01 00:00:00
2012-01-01 00:00:00


In [0]:
# YJ
## target 길이 비교
## test data가 prediction_length만큼 더 김
print (train_entry['target'].shape)
print (test_entry['target'].shape)

(21044,)
(21068,)


In [0]:
# YJ
## target value 비교
print (train_entry['target'][:10])
print (test_entry['target'][:10])
print ()
print (train_entry['target'][-10:])
print (test_entry['target'][-58:-48])
print (test_entry['target'][-10:])

[14. 18. 21. 20. 22. 20. 20. 20. 13. 11.]
[14. 18. 21. 20. 22. 20. 20. 20. 13. 11.]

[6. 4. 4. 2. 2. 4. 8. 6. 9. 7.]
[4. 2. 2. 2. 4. 6. 6. 7. 4. 8.]
[6. 2. 2. 2. 4. 2. 6. 8. 8. 6.]


In [0]:
# YJ
## 'feat_Static_cat' 비교
## 'feat_static_cat' filed는 일단 여기서 무시하고 넘어가고 나중에 다루겠다 함
print (train_entry['feat_static_cat'])
print (test_entry['feat_static_cat'])

[0]
[0]


In [0]:
# YJ
## 'source' filed 비교
## source는 데이터가 저장되어 있는 경로에 대한 정보를 담고 있는 것 같음
print (train_entry['source'])
print (test_entry['source'])

SourceContext(source=PosixPath('/root/.mxnet/gluon-ts/datasets/electricity/train/data.json'), row=0)
SourceContext(source=PosixPath('/root/.mxnet/gluon-ts/datasets/electricity/test/data.json'), row=0)


In [0]:
test_it = iter(dataset.test)

In [0]:
# get the first time series in the test set
test_entry = next(iter(dataset.test))
test_entry.keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# YJ
## test entry 개수(데이터 개수) 확인
### train entry 개수(414)와 같음
target_list = []
for test_entry in iter(dataset.test):
  target_list.append(test_entry['target'][:10])

In [0]:
print (len(target_list))

2247


In [0]:
# YJ
## test entry 개수(데이터 개수) 확인
### train entry 개수(414)와 같음
target_list = []
for test_entry in iter(dataset.test):
  target = str(test_entry['target'][:10])
  print (type(target))
  if target not in target_list:
    target_list.append(target)

In [0]:
len(target_list)

321

In [0]:
# YJ
## test entry 개수(데이터 개수) 확인
### train entry 개수(414)와 같음
target_shape = []
for test_entry in iter(dataset.test):
  target_shape.append(test_entry['target'].shape)

In [0]:
set(target_shape)

{(21068,), (21092,), (21116,), (21140,), (21164,), (21188,), (21212,)}

test_data가 1일(24시간), 2일, 3일, 4일, 5일, 6일, 7일로 되어 있어서 train data 개수(321)보다 7배 더 많음

In [0]:
print (dataset.metadata)

freq='1H' target=None feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=24


In [0]:
dataset.metadata.feat_static_cat

[CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')]

## Estimator 정의

In [0]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.distribution.gaussian import GaussianOutput
from gluonts.trainer import Trainer

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepar_trainer=Trainer(
    ctx = 'gpu',
    epochs=100, # Number of epochs that the network will train (default: 100).
    batch_size=64,# Number of examples in each batch (default: 32).
    num_batches_per_epoch=78, # Number of batches at each epoch (default: 50).
    learning_rate=1e-3, # Initial learning rate (default:  1e−3 ).
    learning_rate_decay_factor=0.1, # Factor (between 0 and 1) by which to decrease the learning rate (default: 0.5).
    patience = 30, # The patience to observe before reducing the learning rate, nonnegative integer (default: 10).
    minimum_learning_rate=1e-6, #  Lower bound for the learning rate (default:  5⋅10−5 ).
    clip_gradient = 10, # Maximum value of gradient. The gradient is clipped if it is too large (default: 10).
    weight_decay=1e-8, # The weight decay (or L2 regularization) coefficient. Modifies objective by adding a penalty for having large weights (default  10−8 ).
    init='xavier', # Initializer of the weights of the network (default: “xavier”).
    hybridize=False, ## ???????
    )

hybridize 관련해서는 이슈가 있는 듯
- https://github.com/awslabs/gluon-ts/issues/833


In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
estimator = DeepAREstimator(
    freq=dataset.metadata.freq, # Frequency of the data to train on and predict
    prediction_length=dataset.metadata.prediction_length, # Length of the prediction horizon
    trainer = deepar_trainer, # Trainer object to be used (default: Trainer())
    context_length=dataset.metadata.prediction_length, # Number of steps to unroll the RNN for before computing predictions (default: None, in which case context_length = prediction_length)
    num_layers=3, # Number of RNN layers (default: 2)
    num_cells=40, # Number of RNN cells for each layer (default: 40)
    cell_type='lstm', # Type of recurrent cells to use (available: ‘lstm’ or ‘gru’; default: ‘lstm’)
    dropout_rate=0.1, # Dropout regularization parameter (default: 0.1)
    use_feat_dynamic_real=False, # Whether to use the feat_dynamic_real field from the data (default: False)
    use_feat_static_cat=True, # Whether to use the feat_static_cat field from the data (default: False)
    use_feat_static_real=False, # Whether to use the feat_static_real field from the data (default: False)
    cardinality=[321], # Number of values of each categorical feature. This must be set if use_feat_static_cat == True (default: None)
    embedding_dimension=[20], # Dimension of the embeddings for categorical features (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = GaussianOutput(), # Distribution to use to evaluate observations and sample predictions (default: StudentTOutput())
    scaling=True, # Whether to automatically scale the target values (default: true)
    lags_seq=None, # Indices of the lagged target values to use as inputs of the RNN (default: None, in which case these are automatically determined based on freq)
    time_features=None, # Time features to use as inputs of the RNN (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=1024, # Number of evaluation samples per time series to increase parallelism during inference. This is a model optimization that does not affect the accuracy (default: 100)
)

In [0]:
predictor = estimator.train(dataset.train)

  0%|          | 0/78 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 78/78 [01:13<00:00,  1.06it/s, epoch=100/100, avg_epoch_loss=5.15]


In [0]:
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.train,  # test dataset
    predictor=predictor,  # predictor
    num_samples=300,  # number of sample paths we want for evaluation
)

In [0]:
# dataset에 있는 데이터 개수 만큼이 만들어짐
forecasts = list(forecast_it)
tss = list(ts_it)

In [0]:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(dataset.train))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 3860.29it/s]


In [0]:
# sampling 결과다 보니, sample paths를 다시 만들 때마다 값이 조금씩 달라지기는 함
## 어느 시점의 값을 결과로 사용해야 하는 걸까???????????
print(json.dumps(agg_metrics, indent=4))

{
    "MSE": 447648.2037265956,
    "abs_error": 1044650.9386005402,
    "abs_target_sum": 16669730.0,
    "abs_target_mean": 2163.7759605399806,
    "seasonal_error": 189.65640599692924,
    "MASE": 0.7864342682619653,
    "MAPE": 0.08320297333314597,
    "sMAPE": 0.11279543998530832,
    "OWA": NaN,
    "MSIS": 7.792327860451494,
    "QuantileLoss[0.1]": 545700.8693288207,
    "Coverage[0.1]": 0.03361889927310486,
    "QuantileLoss[0.5]": 1044650.9407185093,
    "Coverage[0.5]": 0.38356697819314667,
    "QuantileLoss[0.9]": 578568.6512193321,
    "Coverage[0.9]": 0.8783748701973004,
    "RMSE": 669.0651715091705,
    "NRMSE": 0.30921185174004895,
    "ND": 0.06266753802254386,
    "wQuantileLoss[0.1]": 0.03273603527644543,
    "wQuantileLoss[0.5]": 0.06266753814959866,
    "wQuantileLoss[0.9]": 0.03470773979058642,
    "mean_wQuantileLoss": 0.04337043773887683,
    "MAE_Coverage": 0.0681464174454827
}


hybridze=True인 경우

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepar_trainer2=Trainer(
    ctx = 'gpu',
    epochs=100, # Number of epochs that the network will train (default: 100).
    batch_size=256,# Number of examples in each batch (default: 32).
    num_batches_per_epoch=20, # Number of batches at each epoch (default: 50).
    learning_rate=1e-2, # Initial learning rate (default:  1e−3 ).
    learning_rate_decay_factor=0.5, # Factor (between 0 and 1) by which to decrease the learning rate (default: 0.5).
    patience = 20, # The patience to observe before reducing the learning rate, nonnegative integer (default: 10).
    minimum_learning_rate=1e-6, #  Lower bound for the learning rate (default:  5⋅10−5 ).
    clip_gradient = 10, # Maximum value of gradient. The gradient is clipped if it is too large (default: 10).
    weight_decay=1e-8, # The weight decay (or L2 regularization) coefficient. Modifies objective by adding a penalty for having large weights (default  10−8 ).
    init='xavier', # Initializer of the weights of the network (default: “xavier”).
    hybridize=False, ## ???????
    )

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
estimator2 = DeepAREstimator(
    freq=dataset.metadata.freq, # Frequency of the data to train on and predict
    prediction_length=dataset.metadata.prediction_length, # Length of the prediction horizon
    trainer = deepar_trainer2, # Trainer object to be used (default: Trainer())
    context_length=dataset.metadata.prediction_length, # Number of steps to unroll the RNN for before computing predictions (default: None, in which case context_length = prediction_length)
    num_layers=3, # Number of RNN layers (default: 2)
    num_cells=40, # Number of RNN cells for each layer (default: 40)
    cell_type='lstm', # Type of recurrent cells to use (available: ‘lstm’ or ‘gru’; default: ‘lstm’)
    dropout_rate=0.1, # Dropout regularization parameter (default: 0.1)
    use_feat_dynamic_real=False, # Whether to use the feat_dynamic_real field from the data (default: False)
    use_feat_static_cat=True, # Whether to use the feat_static_cat field from the data (default: False)
    use_feat_static_real=False, # Whether to use the feat_static_real field from the data (default: False)
    cardinality=[321], # Number of values of each categorical feature. This must be set if use_feat_static_cat == True (default: None)
    embedding_dimension=[20], # Dimension of the embeddings for categorical features (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = GaussianOutput(), # Distribution to use to evaluate observations and sample predictions (default: StudentTOutput())
    scaling=True, # Whether to automatically scale the target values (default: true)
    lags_seq=None, # Indices of the lagged target values to use as inputs of the RNN (default: None, in which case these are automatically determined based on freq)
    time_features=None, # Time features to use as inputs of the RNN (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=1024, # Number of evaluation samples per time series to increase parallelism during inference. This is a model optimization that does not affect the accuracy (default: 100)
)

In [0]:
predictor2 = estimator2.train(dataset.train)

100%|██████████| 20/20 [00:44<00:00,  2.21s/it, epoch=100/100, avg_epoch_loss=5.15]


In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.train,  # test dataset
    predictor=predictor2,  # predictor
    num_samples=300,  # number of sample paths we want for evaluation
)

In [0]:
# dataset에 있는 데이터 개수 만큼이 만들어짐
forecasts = list(forecast_it)
tss = list(ts_it)

In [0]:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(dataset.train))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 3336.65it/s]


In [0]:
# sampling 결과다 보니, sample paths를 다시 만들 때마다 값이 조금씩 달라지기는 함
## 어느 시점의 값을 결과로 사용해야 하는 걸까???????????
print(json.dumps(agg_metrics, indent=4))

{
    "MSE": 974462.2403720074,
    "abs_error": 1158564.3923244476,
    "abs_target_sum": 16669730.0,
    "abs_target_mean": 2163.7759605399806,
    "seasonal_error": 189.65640599692924,
    "MASE": 0.7406150553997146,
    "MAPE": 0.09075030721376187,
    "sMAPE": 0.12211189502771007,
    "OWA": NaN,
    "MSIS": 7.301125256926607,
    "QuantileLoss[0.1]": 469995.99455171084,
    "Coverage[0.1]": 0.08372274143302186,
    "QuantileLoss[0.5]": 1158564.381817538,
    "Coverage[0.5]": 0.5507528556593979,
    "QuantileLoss[0.9]": 603593.3619128703,
    "Coverage[0.9]": 0.9048546209761166,
    "RMSE": 987.1485401762023,
    "NRMSE": 0.45621568876745205,
    "ND": 0.06950108923926468,
    "wQuantileLoss[0.1]": 0.028194577509756357,
    "wQuantileLoss[0.5]": 0.06950108860896595,
    "wQuantileLoss[0.9]": 0.03620894651040361,
    "mean_wQuantileLoss": 0.044634870876375306,
    "MAE_Coverage": 0.02396157840083087
}


DeepFactor 실험 

In [0]:
import math

from mxnet.gluon import HybridBlock
from mxnet.gluon import nn

# First-party imports
from gluonts.block.feature import FeatureEmbedder
from gluonts.core.component import validated
from gluonts.model.common import Tensor


class DeepFactorNetworkBase(HybridBlock):
    def __init__(
        self,
        global_model: HybridBlock,
        local_model: HybridBlock,
        embedder: FeatureEmbedder,
        **kwargs,
    ) -> None:
        super().__init__(**kwargs)

        self.global_model = global_model
        self.local_model = local_model
        self.embedder = embedder
        with self.name_scope():
            self.loading = nn.Dense(
                units=global_model.num_output, use_bias=False
            )

    def assemble_features(
        self,
        F,
        feat_static_cat: Tensor,  # (batch_size, 1)
        time_feat: Tensor,  # (batch_size, history_length, num_features)
    ) -> Tensor:  # (batch_size, history_length, num_features)
        # todo: this is shared by more than one places, and should be a general routine

        embedded_cat = self.embedder(
            feat_static_cat
        )  # (batch_size, num_features * embedding_size)

        # a workaround when you wish to repeat without knowing the number
        # of repeats
        helper_ones = F.ones_like(
            F.slice_axis(time_feat, axis=2, begin=-1, end=None)
        )
        # (batch_size, history_length, num_features * embedding_size)
        repeated_cat = F.batch_dot(
            helper_ones, F.expand_dims(embedded_cat, axis=1)
        )

        # putting together all the features
        input_feat = F.concat(repeated_cat, time_feat, dim=2)
        return embedded_cat, input_feat

    def compute_global_local(
        self,
        F,
        feat_static_cat: Tensor,  # (batch_size, 1)
        time_feat: Tensor,  # (batch_size, history_length, num_features)
    ) -> (Tensor, Tensor):  # both of size (batch_size, history_length, 1)

        cat, local_input = self.assemble_features(
            F, feat_static_cat, time_feat
        )
        loadings = self.loading(cat)  # (batch_size, num_factors)
        global_factors = self.global_model(
            time_feat
        )  # (batch_size, history_length, num_factors)

        fixed_effect = F.batch_dot(
            global_factors, loadings.expand_dims(axis=2)
        )  # (batch_size, history_length, 1)
        random_effect = F.log(
            F.exp(self.local_model(local_input)) + 1.0
        )  # (batch_size, history_length, 1)
        return F.exp(fixed_effect), random_effect

    def hybrid_forward(self, F, x, *args, **kwargs):
        raise NotImplementedError

    def negative_normal_likelihood(self, F, y, mu, sigma):
        return (
            F.log(sigma)
            + 0.5 * math.log(2 * math.pi)
            + 0.5 * F.square((y - mu) / sigma)
        )


class DeepFactorTrainingNetwork(DeepFactorNetworkBase):
    def hybrid_forward(
        self,
        F,
        feat_static_cat: Tensor,  # (batch_size, 1)
        past_time_feat: Tensor,
        # (batch_size, history_length, num_features)
        past_target: Tensor,  # (batch_size, history_length)
    ) -> Tensor:
        """
        Parameters
        ----------
        F
            Function space
        feat_static_cat
            Shape: (batch_size, 1)
        past_time_feat
            Shape: (batch_size, history_length, num_features)
        past_target
            Shape: (batch_size, history_length)

        Returns
        -------
        Tensor
            A batch of negative log likelihoods.
        """

        fixed_effect, random_effect = self.compute_global_local(
            F, feat_static_cat, past_time_feat
        )

        loss = self.negative_normal_likelihood(
            F, past_target.expand_dims(axis=2), fixed_effect, random_effect
        )
        return loss


class DeepFactorPredictionNetwork(DeepFactorNetworkBase):
    @validated()
    def __init__(
        self, prediction_len: int, num_parallel_samples: int, **kwargs
    ) -> None:
        super().__init__(**kwargs)
        self.prediction_len = prediction_len
        self.num_parallel_samples = num_parallel_samples

    def hybrid_forward(
        self,
        F,
        feat_static_cat: Tensor,
        past_time_feat: Tensor,
        future_time_feat: Tensor,
        past_target: Tensor,
    ) -> Tensor:
        """
        Parameters
        ----------
        F
            Function space
        feat_static_cat
            Shape: (batch_size, 1)
        past_time_feat
            Shape: (batch_size, history_length, num_features)
        future_time_feat
            Shape: (batch_size, prediction_length, num_features)
        past_target
            Shape: (batch_size, history_length)

        Returns
        -------
        Tensor
            Samples of shape (batch_size, num_samples, prediction_length).
        """
        time_feat = F.concat(past_time_feat, future_time_feat, dim=1)
        fixed_effect, random_effect = self.compute_global_local(
            F, feat_static_cat, time_feat
        )

        samples = F.concat(
            *[
                F.sample_normal(fixed_effect, random_effect)
                for _ in range(self.num_parallel_samples)
            ],
            dim=2,
        )  # (batch_size, train_len + prediction_len, num_samples)
        pred_samples = F.slice_axis(
            samples, axis=1, begin=-self.prediction_len, end=None
        )  # (batch_size, prediction_len, num_samples)

        return pred_samples.swapaxes(1, 2)

In [0]:
from typing import List, Optional

# First-party imports
from gluonts import transform
from gluonts.block.feature import FeatureEmbedder
from gluonts.core.component import validated
from gluonts.dataset.field_names import FieldName
from gluonts.distribution import DistributionOutput, StudentTOutput

from gluonts.model.deep_factor.RNNModel import RNNModel
from gluonts.model.deep_factor._network import (
    DeepFactorTrainingNetwork,
    DeepFactorPredictionNetwork,
)
from gluonts.model.estimator import GluonEstimator
from gluonts.model.predictor import Predictor, RepresentableBlockPredictor
from gluonts.time_feature import time_features_from_frequency_str
from gluonts.trainer import Trainer
from gluonts.transform import (
    AddTimeFeatures,
    AsNumpyArray,
    Chain,
    SetFieldIfNotPresent,
    TestSplitSampler,
    Transformation,
    ExpectedNumInstanceSampler,
)


# Third-party imports

class custom_DeepFactorEstimator(GluonEstimator):
    """
    DeepFactorEstimator is an implementation of the 2019 ICML paper "Deep Factors for Forecasting"
    https://arxiv.org/abs/1905.12417.  It uses a global RNN model to learn patterns across multiple related time series
    and an arbitrary local model to model the time series on a per time series basis.  In the current implementation,
    the local model is a RNN (DF-RNN).

    Parameters
    ----------
    freq
        Time series frequency.
    prediction_length
        Prediction length.
    num_hidden_global
        Number of units per hidden layer for the global RNN model (default: 50).
    num_layers_global
        Number of hidden layers for the global RNN model (default: 1).
    num_factors
        Number of global factors (default: 10).
    num_hidden_local
        Number of units per hidden layer for the local RNN model (default: 5).
    num_layers_local
        Number of hidden layers for the global local model (default: 1).
    cell_type
        Type of recurrent cells to use (available: 'lstm' or 'gru';
        default: 'lstm').
    trainer
        Trainer object to be used (default: Trainer()).
    context_length
        Training length (default: None, in which case context_length = prediction_length).
    num_parallel_samples
        Number of evaluation samples per time series to increase parallelism during inference.
        This is a model optimization that does not affect the accuracy (default: 100).
    cardinality
        List consisting of the number of time series (default: list([1]).
    embedding_dimension
        Dimension of the embeddings for categorical features (the same
        dimension is used for all embeddings, default: 10).
    distr_output
        Distribution to use to evaluate observations and sample predictions
        (default: StudentTOutput()).
    """

    @validated()
    def __init__(
        self,
        freq: str,
        prediction_length: int,
        num_hidden_global: int = 50,
        num_layers_global: int = 1,
        num_factors: int = 10,
        num_hidden_local: int = 5,
        num_layers_local: int = 1,
        cell_type: str = "lstm",
        trainer: Trainer = Trainer(),
        context_length: Optional[int] = None,
        num_parallel_samples: int = 100,
        cardinality: List[int] = list([1]),
        embedding_dimension: int = 10,
        distr_output: DistributionOutput = StudentTOutput(),
    ) -> None:
        super().__init__(trainer=trainer)

        assert (
            prediction_length > 0
        ), "The value of `prediction_length` should be > 0"
        assert (
            context_length is None or context_length > 0
        ), "The value of `context_length` should be > 0"
        assert num_layers_global > 0, "The value of `num_layers` should be > 0"
        assert num_hidden_global > 0, "The value of `num_hidden` should be > 0"
        assert num_factors > 0, "The value of `num_factors` should be > 0"
        assert (
            num_hidden_local > 0
        ), "The value of `num_hidden_local` should be > 0"
        assert (
            num_layers_local > 0
        ), "The value of `num_layers_local` should be > 0"
        assert all(
            [c > 0 for c in cardinality]
        ), "Elements of `cardinality` should be > 0"
        assert (
            embedding_dimension > 0
        ), "The value of `embedding_dimension` should be > 0"
        assert (
            num_parallel_samples > 0
        ), "The value of `num_parallel_samples` should be > 0"

        self.freq = freq
        self.context_length = (
            context_length if context_length is not None else prediction_length
        )
        self.prediction_length = prediction_length
        self.distr_output = distr_output
        self.num_parallel_samples = num_parallel_samples
        self.cardinality = cardinality
        self.embedding_dimensions = [embedding_dimension for _ in cardinality]

        self.global_model = RNNModel(
            mode=cell_type,
            num_hidden=num_hidden_global,
            num_layers=num_layers_global,
            num_output=num_factors,
        )

        # TODO: Allow the local model to be defined as an arbitrary local model, e.g. DF-GP and DF-LDS
        self.local_model = RNNModel(
            mode=cell_type,
            num_hidden=num_hidden_local,
            num_layers=num_layers_local,
            num_output=1,
        )

    def create_transformation(self) -> Transformation:
        return Chain(
            trans=[
                AsNumpyArray(field=FieldName.TARGET, expected_ndim=1),
                AddTimeFeatures(
                    start_field=FieldName.START,
                    target_field=FieldName.TARGET,
                    output_field=FieldName.FEAT_TIME,
                    time_features=time_features_from_frequency_str(self.freq),
                    pred_length=self.prediction_length,
                ),
                SetFieldIfNotPresent(
                    field=FieldName.FEAT_STATIC_CAT, value=[0.0]
                ),
                AsNumpyArray(field=FieldName.FEAT_STATIC_CAT, expected_ndim=1),
                transform.InstanceSplitter(
                    target_field=FieldName.TARGET,
                    is_pad_field=FieldName.IS_PAD,
                    start_field=FieldName.START,
                    forecast_start_field=FieldName.FORECAST_START,
                    train_sampler=ExpectedNumInstanceSampler(num_instances=1),
                    #train_sampler=TestSplitSampler(),
                    time_series_fields=[FieldName.FEAT_TIME],
                    past_length=self.context_length,
                    future_length=self.prediction_length,
                ),
            ]
        )

    def create_training_network(self) -> DeepFactorTrainingNetwork:
        return DeepFactorTrainingNetwork(
            embedder=FeatureEmbedder(
                cardinalities=self.cardinality,
                embedding_dims=self.embedding_dimensions,
            ),
            global_model=self.global_model,
            local_model=self.local_model,
        )

    def create_predictor(
        self,
        transformation: Transformation,
        trained_network: DeepFactorTrainingNetwork,
    ) -> Predictor:
        prediction_net = DeepFactorPredictionNetwork(
            embedder=trained_network.embedder,
            global_model=trained_network.global_model,
            local_model=trained_network.local_model,
            prediction_len=self.prediction_length,
            num_parallel_samples=self.num_parallel_samples,
            params=trained_network.collect_params(),
        )

        return RepresentableBlockPredictor(
            input_transform=transformation,
            prediction_net=prediction_net,
            batch_size=self.trainer.batch_size,
            freq=self.freq,
            prediction_length=self.prediction_length,
            ctx=self.trainer.ctx,
        )

In [0]:
#from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.distribution.student_t import StudentTOutput

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepfactor_trainer=Trainer(
    ctx = 'gpu',
    epochs=100, # Number of epochs that the network will train (default: 100).
    batch_size=64,# Number of examples in each batch (default: 32).
    num_batches_per_epoch=78, # Number of batches at each epoch (default: 50).
    learning_rate=1e-3, # Initial learning rate (default:  1e−3 ).
    learning_rate_decay_factor=0.5, # Factor (between 0 and 1) by which to decrease the learning rate (default: 0.5).
    patience = 10, # The patience to observe before reducing the learning rate, nonnegative integer (default: 10).
    minimum_learning_rate=5e-5, #  Lower bound for the learning rate (default:  5⋅1e−5 ).
    clip_gradient = 10, # Maximum value of gradient. The gradient is clipped if it is too large (default: 10).
    weight_decay=1e-8, # The weight decay (or L2 regularization) coefficient. Modifies objective by adding a penalty for having large weights (default  10−8 ).
    init='xavier', # Initializer of the weights of the network (default: “xavier”).
    hybridize=True, ## ???????
    )

In [0]:
# https://gluon-ts.mxnet.io/_modules/gluonts/model/deep_factor/_estimator.html#DeepFactorEstimator
DF_estimator = custom_DeepFactorEstimator(
    freq=dataset.metadata.freq, # Time series frequency
    prediction_length=dataset.metadata.prediction_length, # Prediction length.
    num_hidden_global=50, # Number of units per hidden layer for the global RNN model (default: 50).
    num_layers_global=1, # Number of hidden layers for the global RNN model (default: 1).
    num_factors=10, # Number of global factors (default: 10).
    num_hidden_local=5, # Number of units per hidden layer for the local RNN model (default: 5).
    num_layers_local=1, # Number of hidden layers for the global local model (default: 1).
    cell_type='lstm', # Type of recurrent cells to use (available: ‘lstm’ or ‘gru’; default: ‘lstm’).
    trainer = deepfactor_trainer, # Trainer object to be used (default: Trainer()).
    context_length=None, # Training length (default: None, in which case context_length = prediction_length).
    num_parallel_samples=1024, # Number of evaluation samples per time series to increase parallelism during inference. This is a model optimization that does not affect the accuracy (default: 100).
    cardinality=[1], # List consisting of the number of time series (default: list([1]).
    embedding_dimension=10, # Dimension of the embeddings for categorical features (the same dimension is used for all embeddings, default: 10).
    distr_output=StudentTOutput(), # Distribution to use to evaluate observations and sample predictions (default: StudentTOutput()).
)

In [0]:
DF_predictor = DF_estimator.train(dataset.train)

  0%|          | 0/78 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


 38%|███▊      | 30/78 [00:10<00:16,  2.98it/s, epoch=62/100, avg_epoch_loss=9.02e+5]

In [0]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.train,  # test dataset
    predictor=DF_predictor,  # predictor
    num_samples=300,  # number of sample paths we want for evaluation
)

In [0]:
# dataset에 있는 데이터 개수 만큼이 만들어짐
forecasts = list(forecast_it)
tss = list(ts_it)

In [0]:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(dataset.train))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 3646.95it/s]


In [0]:
# sampling 결과다 보니, sample paths를 다시 만들 때마다 값이 조금씩 달라지기는 함
## 어느 시점의 값을 결과로 사용해야 하는 걸까???????????
print(json.dumps(agg_metrics, indent=4))

{
    "MSE": 125489484.04337032,
    "abs_error": 16843198.954589844,
    "abs_target_sum": 16669730.0,
    "abs_target_mean": 2163.7759605399806,
    "seasonal_error": 189.65640599692924,
    "MASE": 36.17634592559786,
    "MAPE": 6.5716509960768015,
    "sMAPE": 1.0317444022111208,
    "OWA": NaN,
    "MSIS": 1367.7016299807576,
    "QuantileLoss[0.1]": 10851790.0078125,
    "Coverage[0.1]": 0.7403946002076843,
    "QuantileLoss[0.5]": 16843198.8069458,
    "Coverage[0.5]": 0.751817237798546,
    "QuantileLoss[0.9]": 22081073.529064942,
    "Coverage[0.9]": 0.7599948078920042,
    "RMSE": 11202.208891257578,
    "NRMSE": 5.177157476350747,
    "ND": 1.010406224611307,
    "wQuantileLoss[0.1]": 0.6509877489204984,
    "wQuantileLoss[0.5]": 1.0104062157542923,
    "wQuantileLoss[0.9]": 1.3246209464139456,
    "mean_wQuantileLoss": 0.9953383036962453,
    "MAE_Coverage": 0.34407234337140874
}
